In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "prototyping"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using JLD2
using GLMakie
using GeometryBasics
using PyCall
using Distances

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [ ]:
function get_barycentric_subdivision_and_filtration_old(points, mc_tets, n_atoms_per_mol)
    barycenters = Vector{Point3f}([])
    filtration = Vector{Tuple{Vector{Int}, Float32}}([])
    total_vertices = 0
    for vs in eachrow(mc_tets)
        part_one = [v+1 for v in vs if div(v, n_atoms_per_mol)==0]
        part_two = [v+1 for v in vs if div(v, n_atoms_per_mol)==1]
        if length(part_one) == length(part_two) 
            #Get Simplices and Values for even split
            x,y = part_one[1], part_one[2]
            u,v = part_two[1], part_two[2]
            barycenters = [barycenters; [
                get_barycenter(points, [x,y,u,v]),  #1
                get_barycenter(points, [x,v]),      #2
                get_barycenter(points, [x,y,v]),    #3
                get_barycenter(points, [y,v]),      #4
                get_barycenter(points, [y,u,v]),    #5
                get_barycenter(points, [y,u]),      #6
                get_barycenter(points, [x,y,u]),    #7
                get_barycenter(points, [x,u]),      #8
                get_barycenter(points, [x,u,v]),    #9
            ]]
            vertices = [
                ([1] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [u,v])) / 2.0), 
                ([2] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v])) / 2.0),
                ([3] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [v])) / 2.0),
                ([4] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [v])) / 2.0),
                ([5] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [u, v])) / 2.0),
                ([6] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [u])) / 2.0),
                ([7] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [u])) / 2.0),
                ([8] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u])) / 2.0),
                ([9] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v,u])) / 2.0),
            ]
            edges = [
                ([1,2] .+ total_vertices, minimum([vertices[1][2], vertices[2][2]])),
                ([1,3] .+ total_vertices, minimum([vertices[1][2], vertices[3][2]])),
                ([1,4] .+ total_vertices, minimum([vertices[1][2], vertices[4][2]])),
                ([1,5] .+ total_vertices, minimum([vertices[1][2], vertices[5][2]])),
                ([1,6] .+ total_vertices, minimum([vertices[1][2], vertices[6][2]])),
                ([1,7] .+ total_vertices, minimum([vertices[1][2], vertices[7][2]])),
                ([1,8] .+ total_vertices, minimum([vertices[1][2], vertices[8][2]])),
                ([1,9] .+ total_vertices, minimum([vertices[1][2], vertices[9][2]])),
                ([2,3] .+ total_vertices, minimum([vertices[2][2], vertices[3][2]])),
                ([3,4] .+ total_vertices, minimum([vertices[3][2], vertices[4][2]])),
                ([4,5] .+ total_vertices, minimum([vertices[4][2], vertices[5][2]])),
                ([5,6] .+ total_vertices, minimum([vertices[5][2], vertices[6][2]])),
                ([6,7] .+ total_vertices, minimum([vertices[6][2], vertices[7][2]])),
                ([7,8] .+ total_vertices, minimum([vertices[7][2], vertices[8][2]])),
                ([8,9] .+ total_vertices, minimum([vertices[8][2], vertices[9][2]])),
                ([9,2] .+ total_vertices, minimum([vertices[9][2], vertices[2][2]]))
            ]
            triangles = [
                ([1,2,3] .+ total_vertices, minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                ([1,3,4] .+ total_vertices, minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                ([1,4,5] .+ total_vertices, minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                ([1,5,6] .+ total_vertices, minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                ([1,6,7] .+ total_vertices, minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                ([1,7,8] .+ total_vertices, minimum([vertices[1][2], vertices[7][2], vertices[8][2]])),
                ([1,8,9] .+ total_vertices, minimum([vertices[1][2], vertices[8][2], vertices[9][2]])),
                ([1,9,2] .+ total_vertices, minimum([vertices[1][2], vertices[9][2], vertices[2][2]]))
            ]
            total_vertices += 9
            filtration = [filtration; vertices; edges; triangles]
        end
        if length(part_one) != length(part_two)
            if length(part_one) < length(part_two)
                part_one, part_two = part_two, part_one
            end
            u,v,w = part_one[1], part_one[2], part_one[3]
            x = part_two[1]

            barycenters = [barycenters; [
                get_barycenter(points, [u,v,w,x]),  #1
                get_barycenter(points, [x,v]),      #2
                get_barycenter(points, [x,v,w]),    #3
                get_barycenter(points, [x,w]),      #4
                get_barycenter(points, [w,u,x]),    #5
                get_barycenter(points, [u,x]),      #6
                get_barycenter(points, [u,v,x]),    #7
            ]]
            vertices = [
                ([1] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u,v,w])) / 2.0), 
                ([2] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v])) / 2.0),
                ([3] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v,w])) / 2.0),
                ([4] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [w])) / 2.0),
                ([5] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [w,u])) / 2.0),
                ([6] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u])) / 2.0),
                ([7] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u,v])) / 2.0),
            ]
            edges = [
                ([1,2] .+ total_vertices, minimum([vertices[1][2], vertices[2][2]])),
                ([1,3] .+ total_vertices, minimum([vertices[1][2], vertices[3][2]])),
                ([1,4] .+ total_vertices, minimum([vertices[1][2], vertices[4][2]])),
                ([1,5] .+ total_vertices, minimum([vertices[1][2], vertices[5][2]])),
                ([1,6] .+ total_vertices, minimum([vertices[1][2], vertices[6][2]])),
                ([1,7] .+ total_vertices, minimum([vertices[1][2], vertices[7][2]])),
                ([2,3] .+ total_vertices, minimum([vertices[2][2], vertices[3][2]])),
                ([3,4] .+ total_vertices, minimum([vertices[3][2], vertices[4][2]])),
                ([4,5] .+ total_vertices, minimum([vertices[4][2], vertices[5][2]])),
                ([5,6] .+ total_vertices, minimum([vertices[5][2], vertices[6][2]])),
                ([6,7] .+ total_vertices, minimum([vertices[6][2], vertices[7][2]])),
                ([7,2] .+ total_vertices, minimum([vertices[7][2], vertices[2][2]]))
            ]
            triangles = [
                ([1,2,3] .+ total_vertices, minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                ([1,3,4] .+ total_vertices, minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                ([1,4,5] .+ total_vertices, minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                ([1,5,6] .+ total_vertices, minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                ([1,6,7] .+ total_vertices, minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                ([1,7,2] .+ total_vertices, minimum([vertices[1][2], vertices[7][2], vertices[2][2]]))
            ]
            total_vertices += 7
            filtration = [filtration; vertices; edges; triangles]
        end
    end
    barycenters, filtration
end

function get_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)
    barycenters = Vector{Point3f}([])
    filtration = Vector{Tuple{Vector{Int}, Float32}}([])
    total_vertices = 0
    for vs in eachrow(mc_tets)
        part_one = [v+1 for v in vs if div(v, n_atoms_per_mol)==0]
        part_two = [v+1 for v in vs if div(v, n_atoms_per_mol)==1]
        if length(part_one) == length(part_two) 
            #Get Simplices and Values for even split
            x,y = part_one[1], part_one[2]
            u,v = part_two[1], part_two[2]

            bc_e = [
                get_barycenter(points, [x,v]),
                get_barycenter(points, [y,v]),
                get_barycenter(points, [y,u]),
                get_barycenter(points, [x,u])
            ]

            barycenters = [barycenters; [
                get_barycenter(bc_e, [1,2,3,4]),    #u v x y
                bc_e[1],                            #x v
                get_barycenter(bc_e, [1,2]),        #x y v
                bc_e[2],                            #y v
                get_barycenter(bc_e, [2,3]),        #y u v
                bc_e[3],                            #y u
                get_barycenter(bc_e, [3,4]),        #x y u
                bc_e[4],                            #x u
                get_barycenter(bc_e, [1,4]),        #x u v
            ]]
            vertices = [
                ([1] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [u,v])) / 2.0), 
                ([2] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v])) / 2.0),
                ([3] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [v])) / 2.0),
                ([4] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [v])) / 2.0),
                ([5] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [u,v])) / 2.0),
                ([6] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [u])) / 2.0),
                ([7] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [u])) / 2.0),
                ([8] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u])) / 2.0),
                ([9] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v,u])) / 2.0),
            ]
            edges = [
                ([1,2] .+ total_vertices, minimum([vertices[1][2], vertices[2][2]])),
                ([1,3] .+ total_vertices, minimum([vertices[1][2], vertices[3][2]])),
                ([1,4] .+ total_vertices, minimum([vertices[1][2], vertices[4][2]])),
                ([1,5] .+ total_vertices, minimum([vertices[1][2], vertices[5][2]])),
                ([1,6] .+ total_vertices, minimum([vertices[1][2], vertices[6][2]])),
                ([1,7] .+ total_vertices, minimum([vertices[1][2], vertices[7][2]])),
                ([1,8] .+ total_vertices, minimum([vertices[1][2], vertices[8][2]])),
                ([1,9] .+ total_vertices, minimum([vertices[1][2], vertices[9][2]])),
                ([2,3] .+ total_vertices, minimum([vertices[2][2], vertices[3][2]])),
                ([3,4] .+ total_vertices, minimum([vertices[3][2], vertices[4][2]])),
                ([4,5] .+ total_vertices, minimum([vertices[4][2], vertices[5][2]])),
                ([5,6] .+ total_vertices, minimum([vertices[5][2], vertices[6][2]])),
                ([6,7] .+ total_vertices, minimum([vertices[6][2], vertices[7][2]])),
                ([7,8] .+ total_vertices, minimum([vertices[7][2], vertices[8][2]])),
                ([8,9] .+ total_vertices, minimum([vertices[8][2], vertices[9][2]])),
                ([9,2] .+ total_vertices, minimum([vertices[9][2], vertices[2][2]]))
            ]
            triangles = [
                ([1,2,3] .+ total_vertices, minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                ([1,3,4] .+ total_vertices, minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                ([1,4,5] .+ total_vertices, minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                ([1,5,6] .+ total_vertices, minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                ([1,6,7] .+ total_vertices, minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                ([1,7,8] .+ total_vertices, minimum([vertices[1][2], vertices[7][2], vertices[8][2]])),
                ([1,8,9] .+ total_vertices, minimum([vertices[1][2], vertices[8][2], vertices[9][2]])),
                ([1,9,2] .+ total_vertices, minimum([vertices[1][2], vertices[9][2], vertices[2][2]]))
            ]
            total_vertices += 9
            filtration = [filtration; vertices; edges; triangles]
        end
        if length(part_one) != length(part_two)
            if length(part_one) < length(part_two)
                part_one, part_two = part_two, part_one
            end
            u,v,w = part_one[1], part_one[2], part_one[3]
            x = part_two[1]

            bc_e = [
                get_barycenter(points, [x,v]),      #2
                get_barycenter(points, [x,w]),      #4
                get_barycenter(points, [u,x])       #6
            ]

            barycenters = [barycenters; [
                get_barycenter(bc_e, [1,2,3]),  #1
                bc_e[1],                        #2
                get_barycenter(bc_e, [1,2]),    #3
                bc_e[2],                        #4
                get_barycenter(bc_e, [2,3]),    #5
                bc_e[3],                        #6
                get_barycenter(bc_e, [1,3]),    #7
            ]]
            vertices = [
                ([1] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u,v,w])) / 2.0), 
                ([2] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v])) / 2.0),
                ([3] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v,w])) / 2.0),
                ([4] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [w])) / 2.0),
                ([5] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [w,u])) / 2.0),
                ([6] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u])) / 2.0),
                ([7] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u,v])) / 2.0),
            ]
            edges = [
                ([1,2] .+ total_vertices, minimum([vertices[1][2], vertices[2][2]])),
                ([1,3] .+ total_vertices, minimum([vertices[1][2], vertices[3][2]])),
                ([1,4] .+ total_vertices, minimum([vertices[1][2], vertices[4][2]])),
                ([1,5] .+ total_vertices, minimum([vertices[1][2], vertices[5][2]])),
                ([1,6] .+ total_vertices, minimum([vertices[1][2], vertices[6][2]])),
                ([1,7] .+ total_vertices, minimum([vertices[1][2], vertices[7][2]])),
                ([2,3] .+ total_vertices, minimum([vertices[2][2], vertices[3][2]])),
                ([3,4] .+ total_vertices, minimum([vertices[3][2], vertices[4][2]])),
                ([4,5] .+ total_vertices, minimum([vertices[4][2], vertices[5][2]])),
                ([5,6] .+ total_vertices, minimum([vertices[5][2], vertices[6][2]])),
                ([6,7] .+ total_vertices, minimum([vertices[6][2], vertices[7][2]])),
                ([7,2] .+ total_vertices, minimum([vertices[7][2], vertices[2][2]]))
            ]
            triangles = [
                ([1,2,3] .+ total_vertices, minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                ([1,3,4] .+ total_vertices, minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                ([1,4,5] .+ total_vertices, minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                ([1,5,6] .+ total_vertices, minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                ([1,6,7] .+ total_vertices, minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                ([1,7,2] .+ total_vertices, minimum([vertices[1][2], vertices[7][2], vertices[2][2]]))
            ]
            total_vertices += 7
            filtration = [filtration; vertices; edges; triangles]
        end
    end
    barycenters, filtration
end

get_barycentric_subdivision_and_filtration_old (generic function with 1 method)

In [ ]:
function get_multichromatic_tetrahedra(points, n_atoms_per_mol)
    py"""
    import numpy as np
    import diode

    def get_multichromatic_tetrahedra(points, n_atoms_per_mol):        
        def is_multi(sigma):
            return len(set(v // n_atoms_per_mol for v in sigma)) >= 2
        points = np.asarray(points)
        tetrahedra = [vs for (vs,fs) in diode.fill_alpha_shapes(points) if len(vs) == 4 and is_multi(vs)]
        return tetrahedra
    """
    py"get_multichromatic_tetrahedra"(points, n_atoms_per_mol)
end

function get_barycenter(points, vertices) 
    Point3f(sum(points[vertices]) / length(vertices))
end

function get_or_create_bc_simplex_id_and_val!(simplex::Vector{Int}, uob_to_barycenter_simplices, points, n_atoms_per_mol)
    simplex = Tuple(sort(simplex))
    try
        (id, val) = uob_to_barycenter_simplices[simplex]
        return true, (id, val)
    catch
        id = length(uob_to_barycenter_simplices) + 1
        part_one = [v for v in simplex if div(v-1, n_atoms_per_mol)==0]
        part_two = [v for v in simplex if div(v-1, n_atoms_per_mol)==1]
        val = euclidean(get_barycenter(points, part_one), get_barycenter(points, part_two)) / 2.0
        uob_to_barycenter_simplices[simplex] = (id, val)
        return false, (id, val)
    end
end 

function get_no_duplicate_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)
    barycenters = Vector{Point3f}([])
    filtration = Set{Tuple{Vector{Int64}, Float64}}([])
    uob_to_barycenter_simplices = Dict{Any, Any}()
    for vs in eachrow(mc_tets)
        part_one = [v+1 for v in vs if div(v, n_atoms_per_mol)==0]
        part_two = [v+1 for v in vs if div(v, n_atoms_per_mol)==1]
        if length(part_one) == length(part_two) 
            #Get Simplices and Values for even split
            x,y = part_one[1], part_one[2]
            u,v = part_two[1], part_two[2]

            created = Vector{Bool}([])
            vertices = Vector{Tuple{Int, Float64}}([])
            for uob_simplex in [[u,v,x,y], [x,v], [x,y,v], [y,v], [y,u,v], [y,u], [x,y,u], [x,u], [x,u,v]]
                exists, (id, val) = get_or_create_bc_simplex_id_and_val!(uob_simplex, uob_to_barycenter_simplices, points, n_atoms_per_mol)
                push!(vertices, (id, val))
                push!(created, !exists)
            end

            #Placing the points for visualization
            bc_e = [
                get_barycenter(points, [x,v]),
                get_barycenter(points, [y,v]),
                get_barycenter(points, [y,u]),
                get_barycenter(points, [x,u])
            ]

            barycenters = [barycenters; [
                get_barycenter(bc_e, [1,2,3,4]),    #u v x y
                bc_e[1],                            #x v
                get_barycenter(bc_e, [1,2]),        #x y v
                bc_e[2],                            #y v
                get_barycenter(bc_e, [2,3]),        #y u v
                bc_e[3],                            #y u
                get_barycenter(bc_e, [3,4]),        #x y u
                bc_e[4],                            #x u
                get_barycenter(bc_e, [1,4]),        #x u v
            ][created]]

            # We sort edges to filter out duplicates, there will never be dubplicate triangles and vertices dont need to be sorted.
            edges = [
                (sort!([vertices[1][1], vertices[2][1]]), minimum([vertices[1][2], vertices[2][2]])),
                (sort!([vertices[1][1], vertices[3][1]]), minimum([vertices[1][2], vertices[3][2]])),
                (sort!([vertices[1][1], vertices[4][1]]), minimum([vertices[1][2], vertices[4][2]])),
                (sort!([vertices[1][1], vertices[5][1]]), minimum([vertices[1][2], vertices[5][2]])),
                (sort!([vertices[1][1], vertices[6][1]]), minimum([vertices[1][2], vertices[6][2]])),
                (sort!([vertices[1][1], vertices[7][1]]), minimum([vertices[1][2], vertices[7][2]])),
                (sort!([vertices[1][1], vertices[8][1]]), minimum([vertices[1][2], vertices[8][2]])),
                (sort!([vertices[1][1], vertices[9][1]]), minimum([vertices[1][2], vertices[9][2]])),
                (sort!([vertices[2][1], vertices[3][1]]), minimum([vertices[2][2], vertices[3][2]])),
                (sort!([vertices[3][1], vertices[4][1]]), minimum([vertices[3][2], vertices[4][2]])),
                (sort!([vertices[4][1], vertices[5][1]]), minimum([vertices[4][2], vertices[5][2]])),
                (sort!([vertices[5][1], vertices[6][1]]), minimum([vertices[5][2], vertices[6][2]])),
                (sort!([vertices[6][1], vertices[7][1]]), minimum([vertices[6][2], vertices[7][2]])),
                (sort!([vertices[7][1], vertices[8][1]]), minimum([vertices[7][2], vertices[8][2]])),
                (sort!([vertices[8][1], vertices[9][1]]), minimum([vertices[8][2], vertices[9][2]])),
                (sort!([vertices[9][1], vertices[2][1]]), minimum([vertices[9][2], vertices[2][2]]))
            ]

            triangles = [
                ([vertices[1][1], vertices[2][1], vertices[3][1]], minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                ([vertices[1][1], vertices[3][1], vertices[4][1]], minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                ([vertices[1][1], vertices[4][1], vertices[5][1]], minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                ([vertices[1][1], vertices[5][1], vertices[6][1]], minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                ([vertices[1][1], vertices[6][1], vertices[7][1]], minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                ([vertices[1][1], vertices[7][1], vertices[8][1]], minimum([vertices[1][2], vertices[7][2], vertices[8][2]])),
                ([vertices[1][1], vertices[8][1], vertices[9][1]], minimum([vertices[1][2], vertices[8][2], vertices[9][2]])),
                ([vertices[1][1], vertices[9][1], vertices[2][1]], minimum([vertices[1][2], vertices[9][2], vertices[2][2]]))
            ]
            union!(filtration, Set([([v[1]], v[2]) for v in vertices]))
            union!(filtration, Set(edges))
            union!(filtration, Set(triangles))
        end
        if length(part_one) != length(part_two)
            if length(part_one) < length(part_two)
                part_one, part_two = part_two, part_one
            end
            u,v,w = part_one[1], part_one[2], part_one[3]
            x = part_two[1]

            created = Vector{Bool}([])
            vertices = Vector{Tuple{Int, Float64}}([])
            for uob_simplex in [[u,v,w,x], [x,v], [x,w,v], [w,x], [w,u,x], [u,x], [u,v,x]]
                exists, (id, val) = get_or_create_bc_simplex_id_and_val!(uob_simplex, uob_to_barycenter_simplices, points, n_atoms_per_mol)
                push!(vertices, (id, val))
                push!(created, !exists)
            end

            bc_e = [
                get_barycenter(points, [x,v]),      
                get_barycenter(points, [x,w]),      
                get_barycenter(points, [u,x])       
            ]

            barycenters = [barycenters; [
                get_barycenter(bc_e, [1,2,3]),  #u v w x
                bc_e[1],                        #v x
                get_barycenter(bc_e, [1,2]),    #v w x
                bc_e[2],                        #w x
                get_barycenter(bc_e, [2,3]),    #w u x
                bc_e[3],                        #u x
                get_barycenter(bc_e, [1,3]),    #u v x
            ][created]]
            
            # We sort edges to filter out duplicates, there will never be dubplicate triangles and vertices dont need to be sorted.
            edges = [
                (sort!([vertices[1][1], vertices[2][1]]), minimum([vertices[1][2], vertices[2][2]])),
                (sort!([vertices[1][1], vertices[3][1]]), minimum([vertices[1][2], vertices[3][2]])),
                (sort!([vertices[1][1], vertices[4][1]]), minimum([vertices[1][2], vertices[4][2]])),
                (sort!([vertices[1][1], vertices[5][1]]), minimum([vertices[1][2], vertices[5][2]])),
                (sort!([vertices[1][1], vertices[6][1]]), minimum([vertices[1][2], vertices[6][2]])),
                (sort!([vertices[1][1], vertices[7][1]]), minimum([vertices[1][2], vertices[7][2]])),
                (sort!([vertices[2][1], vertices[3][1]]), minimum([vertices[2][2], vertices[3][2]])),
                (sort!([vertices[3][1], vertices[4][1]]), minimum([vertices[3][2], vertices[4][2]])),
                (sort!([vertices[4][1], vertices[5][1]]), minimum([vertices[4][2], vertices[5][2]])),
                (sort!([vertices[5][1], vertices[6][1]]), minimum([vertices[5][2], vertices[6][2]])),
                (sort!([vertices[6][1], vertices[7][1]]), minimum([vertices[6][2], vertices[7][2]])),
                (sort!([vertices[7][1], vertices[2][1]]), minimum([vertices[7][2], vertices[2][2]]))
            ]

            triangles = [
                ([vertices[1][1], vertices[2][1], vertices[3][1]], minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                ([vertices[1][1], vertices[3][1], vertices[4][1]], minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                ([vertices[1][1], vertices[4][1], vertices[5][1]], minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                ([vertices[1][1], vertices[5][1], vertices[6][1]], minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                ([vertices[1][1], vertices[6][1], vertices[7][1]], minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                ([vertices[1][1], vertices[7][1], vertices[2][1]], minimum([vertices[1][2], vertices[7][2], vertices[2][2]]))
            ]

            union!(filtration, Set([([v[1]], v[2]) for v in vertices]))
            union!(filtration, Set(edges))
            union!(filtration, Set(triangles))
        end
    end
    barycenters, sort!(sort!(collect(filtration), by = x -> x[1]), by = x -> length(x[1]))
end

get_no_duplicate_barycentric_subdivision_and_filtration (generic function with 1 method)

In [108]:
points = [Point3f([0.0, 0.0, 0.0]), Point3f([2.0, 0.0, 0.0]), Point3f([1.0, 0.0, 2.4]), Point3f([1.0, 10.0, 1.2])]
n_atoms_per_mol = 2
mc_tets = MorphoMol.Energies.get_multichromatic_tetrahedra(points, n_atoms_per_mol)
bcs, filtration = get_no_duplicate_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)
length(filtration)

33

In [163]:
function get_interface_visualization(points::Vector{Point3f}, n_atoms_per_mol::Int, show_wireframe = false)
    mc_tets = MorphoMol.Energies.get_multichromatic_tetrahedra(points, n_atoms_per_mol)
    bcs, filtration = get_no_duplicate_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)

    max_v = sqrt(maximum([v for (_, v) in filtration]))
    min_v = minimum([v for (_, v) in filtration])
    
    f = Figure(fontsize = 12)
    title_fs = 15
    ms = 15
    cm = :Dark2_8
    cr = (1,8)

    i_sc = LScene(f[1:2, 1:2], show_axis=false, scenekw = (lights = [AmbientLight(RGBf(1.0, 1.0, 1.0))],))
    fs = [TriangleFace(e[1]) for e in filtration if length(e[1]) == 3]
    msh = GeometryBasics.Mesh(bcs, fs)

    clr = [e[2] for e in filtration if length(e[1]) == 1]
    mesh!(i_sc, msh, color=clr, colorrange = (min_v, max_v), colormap = :viridis)
    if show_wireframe
        wireframe!(i_sc, msh, color=:black)
    end
    f
end

function get_interface_and_multicolored_tetrahedron_visualization(points::Vector{Point3f}, n_atoms_per_mol::Int)
    mc_tets = MorphoMol.Energies.get_multichromatic_tetrahedra(points, n_atoms_per_mol)
    bcs, filtration = get_no_duplicate_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)

    get_edges(tet) = [
        (tet[1]+1, tet[2]+1),
        (tet[1]+1, tet[3]+1),
        (tet[1]+1, tet[4]+1),
        (tet[2]+1, tet[3]+1),
        (tet[2]+1, tet[4]+1),
        (tet[3]+1, tet[4]+1)
    ]

    f = Figure(fontsize = 12)
    title_fs = 15
    ms = 15
    cm = :Dark2_8
    cr = (1,8)

    i_sc = LScene(f[1:2, 1:2], show_axis=false, scenekw = (lights = [AmbientLight(RGBf(1.0, 1.0, 1.0))],))
    fs = [TriangleFace(e[1]) for e in filtration if length(e[1]) == 3]
    msh = GeometryBasics.Mesh(bcs, fs)

    max_v = maximum([v for (_, v) in filtration])
    min_v = minimum([v for (_, v) in filtration])

    clr = [e[2] for e in filtration if length(e[1]) == 1]
    mesh!(i_sc, msh, color=clr, colorrange = (min_v, max_v), colormap = :viridis)

    #scatter!(i_sc, bcs, markersize = ms, color=:black)
    # for (i,p) in enumerate(points)
    #     if div(i-1, n_atoms_per_mol) == 0
    #         scatter!(i_sc, [p], markersize = ms, color=:red)
    #     elseif div(i-1, n_atoms_per_mol) == 1
    #         scatter!(i_sc, [p], markersize = ms, color=:blue)
    #     end
    # end

    for tet in eachrow(mc_tets)
        for (i,j) in get_edges(tet)
            if div(i-1, n_atoms_per_mol) != div(j-1, n_atoms_per_mol)
                 #lines!(i_sc, [points[i], points[j]], color=:purple, linewidth=2)
            elseif div(i-1, n_atoms_per_mol) == 0
                lines!(i_sc, [points[i], points[j]], color=:red, linewidth=2)
            elseif div(i-1, n_atoms_per_mol) == 1
                lines!(i_sc, [points[i], points[j]], color=:blue, linewidth=2)
            end
        end
    end
    f
end

function get_debug_interface_visualization(points::Vector{Point3f}, n_atoms_per_mol::Int)
    mc_tets = MorphoMol.Energies.get_multichromatic_tetrahedra(points, n_atoms_per_mol)
    bcs, filtration = get_no_duplicate_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)

    max_v = maximum([v for (_, v) in filtration])
    min_v = minimum([v for (_, v) in filtration])
    
    f = Figure(fontsize = 12)
    title_fs = 15
    ms = 15
    cm = :Dark2_8
    cr = (1,8)

    i_sc = LScene(f[1:2, 1:2], show_axis=false, scenekw = (lights = [AmbientLight(RGBf(1.0, 1.0, 1.0))],))

    fs = [TriangleFace(e[1]) for e in filtration if length(e[1]) == 3]
    msh = GeometryBasics.Mesh(bcs, fs)
    clr = [e[2] for e in filtration if length(e[1]) == 1]
    mesh!(i_sc, msh, color=clr, colorrange = (min_v, max_v), colormap = :viridis)
    for (i, bc) in enumerate(bcs)
        scatter!(i_sc, [bc], color=:black, markersize = ms)
        text!(i_sc, [bc]; text="$(i)", fontsize = 15, color=:red)
    end
    f
end

get_debug_interface_visualization (generic function with 1 method)

In [155]:
points = [Point3f([0.0, 0.0, 0.0]), Point3f([2.0, 0.0, 0.0]), Point3f([1.0, 0.0, 2.4]), Point3f([1.0, 3.0, 1.2]), Point3f([4.0, 2.0, 1.8]), Point3f(2.5, 2.5, 2.5)]
n_atoms_per_mol = 3
#display(get_interface_and_multicolored_tetrahedron_visualization(points, n_atoms_per_mol))
display(get_debug_interface_visualization(points, n_atoms_per_mol))
mc_tets = MorphoMol.Energies.get_multichromatic_tetrahedra(points, n_atoms_per_mol)
bcs, filtration = get_no_duplicate_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)

(Point{3, Float32}[[1.0, 1.5, 1.0], [0.5, 1.5, 0.6], [1.0, 1.5, 0.6], [1.5, 1.5, 0.6], [1.25, 1.5, 1.2], [1.0, 1.5, 1.8000001], [0.75, 1.5, 1.2], [1.625, 1.375, 1.5250001], [1.75, 1.25, 2.45], [2.0, 1.25, 1.85]  …  [1.875, 1.375, 0.925], [1.375, 1.375, 2.125], [2.375, 1.125, 1.675], [2.625, 1.125, 1.075], [3.0, 1.0, 0.9], [2.75, 1.0, 1.5], [2.5, 1.0, 2.1], [2.125, 1.125, 2.275], [2.25, 1.25, 0.9166667], [2.25, 1.25, 0.75]], [([1], 1.5132745504379272), ([2], 1.6911535263061523), ([3], 1.6155494451522827), ([4], 1.6911535263061523), ([5], 1.520690679550171), ([6], 1.6155494451522827), ([7], 1.520690679550171), ([8], 1.4184057712554932), ([9], 1.4585952758789062), ([10], 1.494991660118103)  …  ([14, 16, 17], 1.502289891242981), ([14, 17, 18], 1.502289891242981), ([14, 18, 19], 1.502289891242981), ([14, 19, 9], 1.4585952758789062), ([20, 4, 21], 1.4790199995040894), ([20, 11, 12], 1.5701202154159546), ([20, 12, 4], 1.5701202154159546), ([20, 15, 11], 1.5701202154159546), ([20, 16, 15], 1.5

In [130]:
points = [Point3f([0.0, 0.0, 0.0]), Point3f([2.0, 0.0, 0.0]), Point3f([1.0, 0.0, 2.4]), Point3f([1.0, 10.0, 1.2])]
n_atoms_per_mol = 3
#display(compare_interface_colorings(points, n_atoms_per_mol))
display(get_debug_interface_visualization(points, n_atoms_per_mol))

GLMakie.Screen(...)

In [131]:
points = [Point3f([0.0, 0.0, 0.0]), Point3f([2.0, 0.0, 0.0]), Point3f([1.0, 0.0, 2.4]), Point3f([1.0, 3.0, 1.2]), Point3f([4.0, 2.0, 1.8])]
n_atoms_per_mol = 3
#display(get_interface_and_multicolored_tetrahedron_visualization(points, n_atoms_per_mol))
display(get_debug_interface_visualization(points, n_atoms_per_mol))

GLMakie.Screen(...)

In [164]:
@load "../../Data/hpc_out/2_6r7m/1/11_rwm_ma_2_6r7m.jld2" input output
n_atoms_per_mol = length(input["template_radii"])
points = MorphoMol.Utilities.get_point_realization(output["states"][argmin(output["Es"])], input["template_centers"]);


In [166]:
display(get_interface_visualization(points, n_atoms_per_mol, true))
#display(get_interface_and_multicolored_tetrahedron_visualization(points, n_atoms_per_mol))

GLMakie.Screen(...)

In [140]:
# function get_barycentric_subdivision_and_filtration_old(points, mc_tets, n_atoms_per_mol)
#     barycenters = Vector{Point3f}([])
#     filtration = Vector{Tuple{Vector{Int}, Float32}}([])
#     total_vertices = 0
#     for vs in eachrow(mc_tets)
#         part_one = [v+1 for v in vs if div(v, n_atoms_per_mol)==0]
#         part_two = [v+1 for v in vs if div(v, n_atoms_per_mol)==1]
#         if length(part_one) == length(part_two) 
#             #Get Simplices and Values for even split
#             x,y = part_one[1], part_one[2]
#             u,v = part_two[1], part_two[2]
#             barycenters = [barycenters; [
#                 get_barycenter(points, [x,y,u,v]),  #1
#                 get_barycenter(points, [x,v]),      #2
#                 get_barycenter(points, [x,y,v]),    #3
#                 get_barycenter(points, [y,v]),      #4
#                 get_barycenter(points, [y,u,v]),    #5
#                 get_barycenter(points, [y,u]),      #6
#                 get_barycenter(points, [x,y,u]),    #7
#                 get_barycenter(points, [x,u]),      #8
#                 get_barycenter(points, [x,u,v]),    #9
#             ]]
#             vertices = [
#                 ([1] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [u,v])) / 2.0), 
#                 ([2] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v])) / 2.0),
#                 ([3] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [v])) / 2.0),
#                 ([4] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [v])) / 2.0),
#                 ([5] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [u, v])) / 2.0),
#                 ([6] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [u])) / 2.0),
#                 ([7] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [u])) / 2.0),
#                 ([8] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u])) / 2.0),
#                 ([9] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v,u])) / 2.0),
#             ]
#             edges = [
#                 ([1,2] .+ total_vertices, minimum([vertices[1][2], vertices[2][2]])),
#                 ([1,3] .+ total_vertices, minimum([vertices[1][2], vertices[3][2]])),
#                 ([1,4] .+ total_vertices, minimum([vertices[1][2], vertices[4][2]])),
#                 ([1,5] .+ total_vertices, minimum([vertices[1][2], vertices[5][2]])),
#                 ([1,6] .+ total_vertices, minimum([vertices[1][2], vertices[6][2]])),
#                 ([1,7] .+ total_vertices, minimum([vertices[1][2], vertices[7][2]])),
#                 ([1,8] .+ total_vertices, minimum([vertices[1][2], vertices[8][2]])),
#                 ([1,9] .+ total_vertices, minimum([vertices[1][2], vertices[9][2]])),
#                 ([2,3] .+ total_vertices, minimum([vertices[2][2], vertices[3][2]])),
#                 ([3,4] .+ total_vertices, minimum([vertices[3][2], vertices[4][2]])),
#                 ([4,5] .+ total_vertices, minimum([vertices[4][2], vertices[5][2]])),
#                 ([5,6] .+ total_vertices, minimum([vertices[5][2], vertices[6][2]])),
#                 ([6,7] .+ total_vertices, minimum([vertices[6][2], vertices[7][2]])),
#                 ([7,8] .+ total_vertices, minimum([vertices[7][2], vertices[8][2]])),
#                 ([8,9] .+ total_vertices, minimum([vertices[8][2], vertices[9][2]])),
#                 ([9,2] .+ total_vertices, minimum([vertices[9][2], vertices[2][2]]))
#             ]
#             triangles = [
#                 ([1,2,3] .+ total_vertices, minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
#                 ([1,3,4] .+ total_vertices, minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
#                 ([1,4,5] .+ total_vertices, minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
#                 ([1,5,6] .+ total_vertices, minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
#                 ([1,6,7] .+ total_vertices, minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
#                 ([1,7,8] .+ total_vertices, minimum([vertices[1][2], vertices[7][2], vertices[8][2]])),
#                 ([1,8,9] .+ total_vertices, minimum([vertices[1][2], vertices[8][2], vertices[9][2]])),
#                 ([1,9,2] .+ total_vertices, minimum([vertices[1][2], vertices[9][2], vertices[2][2]]))
#             ]
#             total_vertices += 9
#             filtration = [filtration; vertices; edges; triangles]
#         end
#         if length(part_one) != length(part_two)
#             if length(part_one) < length(part_two)
#                 part_one, part_two = part_two, part_one
#             end
#             u,v,w = part_one[1], part_one[2], part_one[3]
#             x = part_two[1]

#             barycenters = [barycenters; [
#                 get_barycenter(points, [u,v,w,x]),  #1
#                 get_barycenter(points, [x,v]),      #2
#                 get_barycenter(points, [x,v,w]),    #3
#                 get_barycenter(points, [x,w]),      #4
#                 get_barycenter(points, [w,u,x]),    #5
#                 get_barycenter(points, [u,x]),      #6
#                 get_barycenter(points, [u,v,x]),    #7
#             ]]
#             vertices = [
#                 ([1] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u,v,w])) / 2.0), 
#                 ([2] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v])) / 2.0),
#                 ([3] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v,w])) / 2.0),
#                 ([4] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [w])) / 2.0),
#                 ([5] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [w,u])) / 2.0),
#                 ([6] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u])) / 2.0),
#                 ([7] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u,v])) / 2.0),
#             ]
#             edges = [
#                 ([1,2] .+ total_vertices, minimum([vertices[1][2], vertices[2][2]])),
#                 ([1,3] .+ total_vertices, minimum([vertices[1][2], vertices[3][2]])),
#                 ([1,4] .+ total_vertices, minimum([vertices[1][2], vertices[4][2]])),
#                 ([1,5] .+ total_vertices, minimum([vertices[1][2], vertices[5][2]])),
#                 ([1,6] .+ total_vertices, minimum([vertices[1][2], vertices[6][2]])),
#                 ([1,7] .+ total_vertices, minimum([vertices[1][2], vertices[7][2]])),
#                 ([2,3] .+ total_vertices, minimum([vertices[2][2], vertices[3][2]])),
#                 ([3,4] .+ total_vertices, minimum([vertices[3][2], vertices[4][2]])),
#                 ([4,5] .+ total_vertices, minimum([vertices[4][2], vertices[5][2]])),
#                 ([5,6] .+ total_vertices, minimum([vertices[5][2], vertices[6][2]])),
#                 ([6,7] .+ total_vertices, minimum([vertices[6][2], vertices[7][2]])),
#                 ([7,2] .+ total_vertices, minimum([vertices[7][2], vertices[2][2]]))
#             ]
#             triangles = [
#                 ([1,2,3] .+ total_vertices, minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
#                 ([1,3,4] .+ total_vertices, minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
#                 ([1,4,5] .+ total_vertices, minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
#                 ([1,5,6] .+ total_vertices, minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
#                 ([1,6,7] .+ total_vertices, minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
#                 ([1,7,2] .+ total_vertices, minimum([vertices[1][2], vertices[7][2], vertices[2][2]]))
#             ]
#             total_vertices += 7
#             filtration = [filtration; vertices; edges; triangles]
#         end
#     end
#     barycenters, filtration
# end

# function get_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)
#     barycenters = Vector{Point3f}([])
#     filtration = Vector{Tuple{Vector{Int}, Float32}}([])
#     total_vertices = 0
#     for vs in eachrow(mc_tets)
#         part_one = [v+1 for v in vs if div(v, n_atoms_per_mol)==0]
#         part_two = [v+1 for v in vs if div(v, n_atoms_per_mol)==1]
#         if length(part_one) == length(part_two) 
#             #Get Simplices and Values for even split
#             x,y = part_one[1], part_one[2]
#             u,v = part_two[1], part_two[2]

#             bc_e = [
#                 get_barycenter(points, [x,v]),
#                 get_barycenter(points, [y,v]),
#                 get_barycenter(points, [y,u]),
#                 get_barycenter(points, [x,u])
#             ]

#             barycenters = [barycenters; [
#                 get_barycenter(bc_e, [1,2,3,4]),    #u v x y
#                 bc_e[1],                            #x v
#                 get_barycenter(bc_e, [1,2]),        #x y v
#                 bc_e[2],                            #y v
#                 get_barycenter(bc_e, [2,3]),        #y u v
#                 bc_e[3],                            #y u
#                 get_barycenter(bc_e, [3,4]),        #x y u
#                 bc_e[4],                            #x u
#                 get_barycenter(bc_e, [1,4]),        #x u v
#             ]]
#             vertices = [
#                 ([1] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [u,v])) / 2.0), 
#                 ([2] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v])) / 2.0),
#                 ([3] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [v])) / 2.0),
#                 ([4] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [v])) / 2.0),
#                 ([5] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [u,v])) / 2.0),
#                 ([6] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [u])) / 2.0),
#                 ([7] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [u])) / 2.0),
#                 ([8] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u])) / 2.0),
#                 ([9] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v,u])) / 2.0),
#             ]
#             edges = [
#                 ([1,2] .+ total_vertices, minimum([vertices[1][2], vertices[2][2]])),
#                 ([1,3] .+ total_vertices, minimum([vertices[1][2], vertices[3][2]])),
#                 ([1,4] .+ total_vertices, minimum([vertices[1][2], vertices[4][2]])),
#                 ([1,5] .+ total_vertices, minimum([vertices[1][2], vertices[5][2]])),
#                 ([1,6] .+ total_vertices, minimum([vertices[1][2], vertices[6][2]])),
#                 ([1,7] .+ total_vertices, minimum([vertices[1][2], vertices[7][2]])),
#                 ([1,8] .+ total_vertices, minimum([vertices[1][2], vertices[8][2]])),
#                 ([1,9] .+ total_vertices, minimum([vertices[1][2], vertices[9][2]])),
#                 ([2,3] .+ total_vertices, minimum([vertices[2][2], vertices[3][2]])),
#                 ([3,4] .+ total_vertices, minimum([vertices[3][2], vertices[4][2]])),
#                 ([4,5] .+ total_vertices, minimum([vertices[4][2], vertices[5][2]])),
#                 ([5,6] .+ total_vertices, minimum([vertices[5][2], vertices[6][2]])),
#                 ([6,7] .+ total_vertices, minimum([vertices[6][2], vertices[7][2]])),
#                 ([7,8] .+ total_vertices, minimum([vertices[7][2], vertices[8][2]])),
#                 ([8,9] .+ total_vertices, minimum([vertices[8][2], vertices[9][2]])),
#                 ([9,2] .+ total_vertices, minimum([vertices[9][2], vertices[2][2]]))
#             ]
#             triangles = [
#                 ([1,2,3] .+ total_vertices, minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
#                 ([1,3,4] .+ total_vertices, minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
#                 ([1,4,5] .+ total_vertices, minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
#                 ([1,5,6] .+ total_vertices, minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
#                 ([1,6,7] .+ total_vertices, minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
#                 ([1,7,8] .+ total_vertices, minimum([vertices[1][2], vertices[7][2], vertices[8][2]])),
#                 ([1,8,9] .+ total_vertices, minimum([vertices[1][2], vertices[8][2], vertices[9][2]])),
#                 ([1,9,2] .+ total_vertices, minimum([vertices[1][2], vertices[9][2], vertices[2][2]]))
#             ]
#             total_vertices += 9
#             filtration = [filtration; vertices; edges; triangles]
#         end
#         if length(part_one) != length(part_two)
#             if length(part_one) < length(part_two)
#                 part_one, part_two = part_two, part_one
#             end
#             u,v,w = part_one[1], part_one[2], part_one[3]
#             x = part_two[1]

#             bc_e = [
#                 get_barycenter(points, [x,v]),      #2
#                 get_barycenter(points, [x,w]),      #4
#                 get_barycenter(points, [u,x])       #6
#             ]

#             barycenters = [barycenters; [
#                 get_barycenter(bc_e, [1,2,3]),  #1
#                 bc_e[1],                        #2
#                 get_barycenter(bc_e, [1,2]),    #3
#                 bc_e[2],                        #4
#                 get_barycenter(bc_e, [2,3]),    #5
#                 bc_e[3],                        #6
#                 get_barycenter(bc_e, [1,3]),    #7
#             ]]
#             vertices = [
#                 ([1] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u,v,w])) / 2.0), 
#                 ([2] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v])) / 2.0),
#                 ([3] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v,w])) / 2.0),
#                 ([4] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [w])) / 2.0),
#                 ([5] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [w,u])) / 2.0),
#                 ([6] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u])) / 2.0),
#                 ([7] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u,v])) / 2.0),
#             ]
#             edges = [
#                 ([1,2] .+ total_vertices, minimum([vertices[1][2], vertices[2][2]])),
#                 ([1,3] .+ total_vertices, minimum([vertices[1][2], vertices[3][2]])),
#                 ([1,4] .+ total_vertices, minimum([vertices[1][2], vertices[4][2]])),
#                 ([1,5] .+ total_vertices, minimum([vertices[1][2], vertices[5][2]])),
#                 ([1,6] .+ total_vertices, minimum([vertices[1][2], vertices[6][2]])),
#                 ([1,7] .+ total_vertices, minimum([vertices[1][2], vertices[7][2]])),
#                 ([2,3] .+ total_vertices, minimum([vertices[2][2], vertices[3][2]])),
#                 ([3,4] .+ total_vertices, minimum([vertices[3][2], vertices[4][2]])),
#                 ([4,5] .+ total_vertices, minimum([vertices[4][2], vertices[5][2]])),
#                 ([5,6] .+ total_vertices, minimum([vertices[5][2], vertices[6][2]])),
#                 ([6,7] .+ total_vertices, minimum([vertices[6][2], vertices[7][2]])),
#                 ([7,2] .+ total_vertices, minimum([vertices[7][2], vertices[2][2]]))
#             ]
#             triangles = [
#                 ([1,2,3] .+ total_vertices, minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
#                 ([1,3,4] .+ total_vertices, minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
#                 ([1,4,5] .+ total_vertices, minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
#                 ([1,5,6] .+ total_vertices, minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
#                 ([1,6,7] .+ total_vertices, minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
#                 ([1,7,2] .+ total_vertices, minimum([vertices[1][2], vertices[7][2], vertices[2][2]]))
#             ]
#             total_vertices += 7
#             filtration = [filtration; vertices; edges; triangles]
#         end
#     end
#     barycenters, filtration
# end

In [ ]:



# function compare_interface_visualizations(points::Vector{Point3f}, n_atoms_per_mol::Int)
#     mc_tets = MorphoMol.Energies.get_multichromatic_tetrahedra(points, n_atoms_per_mol)
       
#     f = Figure(fontsize = 12)
#     title_fs = 15
#     ms = 15
#     cm = :Dark2_8
#     cr = (1,8)

#     bcs, filtration = get_barycentric_subdivision_and_filtration_variant(points, mc_tets, n_atoms_per_mol)
    
#     max_v = sqrt(maximum([v for (_, v) in filtration]))
#     min_v = minimum([v for (_, v) in filtration])

#     i_sc = LScene(f[1, 1], show_axis=false)
#     fs = [TriangleFace(e[1]) for e in filtration if length(e[1]) == 3]
#     msh = GeometryBasics.Mesh(bcs, fs)

#     clr = [e[2] for e in filtration if length(e[1]) == 1]
#     mesh!(i_sc, msh, color=clr, colorrange = (min_v, max_v), colormap = :viridis)
    
#     bcs, filtration = get_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)
#     iv_sc = LScene(f[1, 2], show_axis=false)
#     fs = [TriangleFace(e[1]) for e in filtration if length(e[1]) == 3]
#     msh = GeometryBasics.Mesh(bcs, fs)

#     clr = [e[2] for e in filtration if length(e[1]) == 1]
#     mesh!(iv_sc, msh, color=clr, colorrange = (min_v, max_v), colormap = :viridis)
#     f
# end

# function compare_interface_and_multicolored_tetrahedron_visualization(points::Vector{Point3f}, n_atoms_per_mol::Int, flat::Bool = false)
#     mc_tets = MorphoMol.Energies.get_multichromatic_tetrahedra(points, n_atoms_per_mol)
    
#     get_edges(tet) = [
#         (tet[1]+1, tet[2]+1),
#         (tet[1]+1, tet[3]+1),
#         (tet[1]+1, tet[4]+1),
#         (tet[2]+1, tet[3]+1),
#         (tet[2]+1, tet[4]+1),
#         (tet[3]+1, tet[4]+1)
#     ]

#     f = Figure(fontsize = 12)
#     title_fs = 15
#     ms = 15
#     cm = :Dark2_8
#     cr = (1,8)

#     i_sc = LScene(f[1, 1], show_axis=false)
#     bcs, filtration = get_barycentric_subdivision_and_filtration_variant(points, mc_tets, n_atoms_per_mol)

#     fs = [TriangleFace(e[1]) for e in filtration if length(e[1]) == 3]
#     msh = GeometryBasics.Mesh(bcs, fs)

#     max_v = maximum([v for (_, v) in filtration])
#     min_v = minimum([v for (_, v) in filtration])

#     clr = [e[2] for e in filtration if length(e[1]) == 1]
#     mesh!(i_sc, msh, color=clr, colorrange = (min_v, max_v), colormap = :viridis)

#     scatter!(i_sc, bcs, markersize = ms, color=:black)
#     for (i,p) in enumerate(points)
#         if div(i-1, n_atoms_per_mol) == 0
#             scatter!(i_sc, [p], markersize = ms, color=:red)
#         elseif div(i-1, n_atoms_per_mol) == 1
#             scatter!(i_sc, [p], markersize = ms, color=:blue)
#         end
#     end

#     for tet in eachrow(mc_tets)
#         for (i,j) in get_edges(tet)
#             if div(i-1, n_atoms_per_mol) != div(j-1, n_atoms_per_mol)
#                 lines!(i_sc, [points[i], points[j]], color=:purple, linewidth=2)
#             elseif div(i-1, n_atoms_per_mol) == 0
#                 lines!(i_sc, [points[i], points[j]], color=:red, linewidth=2)
#             elseif div(i-1, n_atoms_per_mol) == 1
#                 lines!(i_sc, [points[i], points[j]], color=:blue, linewidth=2)
#             end
#         end
#     end

#     i_sc = LScene(f[1, 2], show_axis=false)
#     bcs, filtration = get_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol)

#     fs = [TriangleFace(e[1]) for e in filtration if length(e[1]) == 3]
#     msh = GeometryBasics.Mesh(bcs, fs)

#     max_v = maximum([v for (_, v) in filtration])
#     min_v = minimum([v for (_, v) in filtration])

#     clr = [e[2] for e in filtration if length(e[1]) == 1]
#     mesh!(i_sc, msh, color=clr, colorrange = (min_v, max_v), colormap = :viridis)

#     scatter!(i_sc, bcs, markersize = ms, color=:black)
#     for (i,p) in enumerate(points)
#         if div(i-1, n_atoms_per_mol) == 0
#             scatter!(i_sc, [p], markersize = ms, color=:red)
#         elseif div(i-1, n_atoms_per_mol) == 1
#             scatter!(i_sc, [p], markersize = ms, color=:blue)
#         end
#     end

#     for tet in eachrow(mc_tets)
#         for (i,j) in get_edges(tet)
#             if div(i-1, n_atoms_per_mol) != div(j-1, n_atoms_per_mol)
#                 lines!(i_sc, [points[i], points[j]], color=:purple, linewidth=2)
#             elseif div(i-1, n_atoms_per_mol) == 0
#                 lines!(i_sc, [points[i], points[j]], color=:red, linewidth=2)
#             elseif div(i-1, n_atoms_per_mol) == 1
#                 lines!(i_sc, [points[i], points[j]], color=:blue, linewidth=2)
#             end
#         end
#     end
#     f
# end
